# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')
data_path = '../data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
df.corr()

,Pclass,Age,SibSp,Parch,Fare
Pclass,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,-0.549500,0.096067,0.159651,0.216225,1.000000


In [3]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
operation = ['mean', 'median', 'max', 'min', 'count']

for op in operation:
    df_tmp = df.groupby('Sex')['Age'].agg({'Age_'+op:op}).reset_index()
    df_tmp_1 = df.groupby('Sex')['Fare'].agg({'Fare_'+op:op}).reset_index()
    df = pd.merge(df, df_tmp, on='Sex', how='left')
    df = pd.merge(df, df_tmp_1, on='Sex', how='left')
df_tmp = df.groupby('Sex')['Age'].apply(lambda x: x.mode()[0]).reset_index()
df_tmp.columns = ['Sex', 'Age_mode']
df_tmp_1 = df.groupby('Sex')['Fare'].apply(lambda x: x.mode()[0]).reset_index()
df_tmp_1.columns = ['Sex', 'Fare_mode']
df = pd.merge(df, df_tmp, on='Sex', how='left')
df = pd.merge(df, df_tmp_1, on='Sex', how='left')
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,...,Age_median,Fare_median,Age_max,Fare_max,Age_min,Fare_min,Age_count,Fare_count,Age_mode,Fare_mode
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,...,29.0,10.5,80.0,512.3292,0.42,0.00,453,577,19.0,8.05
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,...,27.0,23.0,63.0,512.3292,0.75,6.75,261,314,24.0,7.75
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,...,27.0,23.0,63.0,512.3292,0.75,6.75,261,314,24.0,7.75
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,...,27.0,23.0,63.0,512.3292,0.75,6.75,261,314,24.0,7.75
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,...,29.0,10.5,80.0,512.3292,0.42,0.00,453,577,19.0,8.05


In [4]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

17 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Age_mean', 'Fare_mean', 'Age_median', 'Fare_median', 'Age_max', 'Fare_max', 'Age_min', 'Fare_min', 'Age_count', 'Fare_count', 'Age_mode', 'Fare_mode']



,Pclass,Age,SibSp,Parch,Fare,Age_mean,Fare_mean,Age_median,Fare_median,Age_max,Fare_max,Age_min,Fare_min,Age_count,Fare_count,Age_mode,Fare_mode
0,3,22.0,1,0,7.2500,30.726645,25.523893,29.0,10.5,80.0,512.3292,0.42,0.00,453,577,19.0,8.05
1,1,38.0,1,0,71.2833,27.915709,44.479818,27.0,23.0,63.0,512.3292,0.75,6.75,261,314,24.0,7.75
2,3,26.0,0,0,7.9250,27.915709,44.479818,27.0,23.0,63.0,512.3292,0.75,6.75,261,314,24.0,7.75
3,1,35.0,1,0,53.1000,27.915709,44.479818,27.0,23.0,63.0,512.3292,0.75,6.75,261,314,24.0,7.75
4,3,35.0,0,0,8.0500,30.726645,25.523893,29.0,10.5,80.0,512.3292,0.42,0.00,453,577,19.0,8.05


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?  
Ans: 透過性別與年齡合併出的新特徵有顯著的改善!!  
     單獨使用新特徵的效果更好

In [5]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
df_ori = df.drop(['Age_mean', 'Age_median', 'Age_max', 'Age_min', 'Age_count', 'Age_mode',
                  'Fare_mean', 'Fare_median', 'Fare_max', 'Fare_min', 'Fare_count', 'Fare_mode'], axis=1)
mm = MinMaxScaler()
train_X = mm.fit_transform(df_ori)
LR_estimator = LogisticRegression()
LR_result = cross_val_score(LR_estimator, train_X, train_Y, scoring='accuracy', cv=5)
print(f'Logistic_Regression Score:{LR_result.mean()}')

Logistic_Regression Score:0.7038635542329971


In [7]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = mm.fit_transform(df)
LR_estimator = LogisticRegression()
LR_result = cross_val_score(LR_estimator, train_X, train_Y, scoring='accuracy', cv=5)
print(f'Logistic_Regression Score:{LR_result.mean()}')

Logistic_Regression Score:0.7833947386806166


In [6]:
# New feature Only
df_new = df[['Age_mean', 'Age_median', 'Age_max', 'Age_min', 'Age_count', 'Age_mode',
                  'Fare_mean', 'Fare_median', 'Fare_max', 'Fare_min', 'Fare_count', 'Fare_mode']]
train_X = mm.fit_transform(df_new)
LR_estimator = LogisticRegression()
LR_result = cross_val_score(LR_estimator, train_X, train_Y, scoring='accuracy', cv=5)
print(f'Logistic_Regression Score:{LR_result.mean()}')

Logistic_Regression Score:0.7867150249291879
